In [10]:
import pandas as pd
import numpy as np
from helper_functions import chart_categorical_comparison

In [11]:
COLOR_DICT = {
                'cdn': '#B32E29',
                'us': '#3279B3',
                'uk': '#DE7012',
                'int': '#FACD14'
             }

In [12]:
df = pd.read_csv('data/clean_data.csv')

In [13]:
df.columns

Index(['Unnamed: 0', 'week_of', 'station', 'station_city', 'station_province',
       'station_latitude', 'station_longitude', 'chart_position',
       'artist_name(s)', 'artist_country', 'artist_home_city',
       'artist_home_latitude', 'artist_home_longitude',
       'km_distance_(home_station)', 'album_name', 'label_name', 'label_type',
       'language_of_music', 'visible_ethnic_minority',
       'census_race_classification', 'artist_gender', 'm_music', 'a_artist',
       'p_performance', 'l_lyrics', 'artist_is_group', 'canadian_content'],
      dtype='object')

In [23]:
chart_categorical_comparison(
                                        categories = df['census_race_classification'], 
                                        compare_by = df['artist_country'], 
                                        comparison_filter=['us','cdn','uk','int'], 
                                        color_dict=COLOR_DICT )

In [24]:
chart_categorical_comparison(
                                        categories = df['artist_gender'], 
                                        compare_by = df['artist_country'], 
                                        comparison_filter=['us','cdn','uk','int'], 
                                        color_dict=COLOR_DICT )

In [29]:
chart_categorical_comparison(
                                        categories = df['census_race_classification'], 
                                        compare_by = df['station_province'], 
                                        # comparison_filter=['us','cdn','uk','int'], 
                                        # color_dict=COLOR_DICT )
                            )   

In [19]:
ontario_df = df[df['station_province'] == 'ontario']
chart_categorical_comparison(
                                        categories = ontario_df['census_race_classification'], 
                                        compare_by = ontario_df['station_city'], 
                                        # comparison_filter=['us','cdn','uk','int'], 
                                        # color_dict=COLOR_DICT )
                            )   